# Binary Tree Cameras
https://leetcode.com/problems/binary-tree-cameras/

## Initial Thoughts
- The problem asks for the minimum number of nodes necessary. We should be able to establish a mathematical upper bound for the minimum without fully traversing the tree
- If we must traverse the tree fully, it is slightly tricky to determine whether a particular node needs to be a camera, since that depends on its parent
- There is no use in any leaf node being a camera (except for the edge case in which it's also the root). In general, putting the camera on the parent will either decrease or keep the same the number of cameras needed.
- The optimal configuration is in general not unique

### Bounding the number of cameras
- Consider the case in which we have a tree that is fully degenerate, i.e. a single branch. A camera can account for up to three nodes. So placing the cameras as spaced out as possible gives us that we need at minimum `MIN_CAMERAS = ceiling(n / 3)` cameras
- When we have branches, each camera can potentially account for up to 4 nodes, so the minimum must be `MIN_CAMERAS >= ceiling(n / 4)` nodes. The degenerate case is the worst case, so in general we have `ceiling(n / 4) <= MIN_CAMERAS <= ceiling(n / 3)`
- This exact number for does depend on the location of these branches. We are going to have to iterate through the tree.

### Iteration plan (constructive)
- If a node is a leaf, it doesn't need a camera. Its parent can handle that (assuming it has a parent).
- Since that's an easy base case let's do DFS.
- If a node has a child that is a camera it is watched.
- If a node has an unwatched child, it now must be a camera (since we are doing DFS)
- If a node has no unwatched children, it does not need to be a camera (unless it is the root)


Let's try it

```python
def yieldCameraLocations(node):
    if node.left is None and node.right is None:
        return

    if node.left is not None:
        # recurse; yield all camera nodes on the left
        # node.left could be
        #     1. a camera
        #     2. watched
        #     3. as-yet unwatched

    # Repeat steps with respect to node.right

    if isEitherChildUnwatched:
        # current node must be a camera
        yield node
        # stop iteration
        return

    if isRoot and not isEitherChildACamera
        # current node must be a camera
        yield node
        # stop iteration
        return
```

Note that there's some holes to fill in. The obvious question is: how do we figure out whether a child node is a camera, watched, or unwatched? A given node knows whether it is a camera, watched, or unwatched, based on the value of its children alone, i.e. a node shouldn't traverse layers deep of its children to find out what state a child is in, since the child already knows. IMO it's a little hacky, but with generators we can both yield values (the known cameras, to be just re-yielded) AND return a value (direct child-to-parent communication). So let's just do that.

In [4]:
# This generator yields all known camera locations and returns a tuple of booleans - isCamera, isWatched
# note - in other languages, an enum would be more clean IMO, since there are really just three states. 
def yieldCameraLocations(node, isRoot = True):
    isEitherChildUnwatched = False
    isEitherChildCamera = False

    if node.left is not None:
        isCamera, isWatched = yield from yieldCameraLocations(node.left, False)

        isEitherChildCamera = isEitherChildCamera or isCamera
        isEitherChildUnwatched = isEitherChildUnwatched or not isWatched

    if node.right is not None:
        isCamera, isWatched = yield from yieldCameraLocations(node.right, False)

        isEitherChildCamera = isEitherChildCamera or isCamera
        isEitherChildUnwatched = isEitherChildUnwatched or not isWatched
    
    if isEitherChildUnwatched:
        yield node
        return True, True

    if isRoot and not isEitherChildCamera:
        yield node
        # Doesn't really matter what we return since no parent is listening
        return

    return False, isEitherChildCamera

If that works, all that's left to do is count the number of nodes that we yield

In [ ]:
def solution(root):
    if root is None:
        return 0
    
    return sum(1 for camera in yieldCameraLocations(root))


## Success
Faster than 83%, less memory than 92%. And we still constructed an optimal solution on the way, in case we're asked for it.

### Complexity analysis

- Time: `O(n)`, with `n` the number of nodes in the tree
- Space: `O(h)`, with `h` the depth of the tree (and thus our call stack)